In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from quspin.basis import spin_basis_1d
from quspin.operators import hamiltonian
from quspin.tools.evolution import ED_state_vs_time, expm_multiply_parallel
from quspin.tools.Floquet import Floquet_t_vec, Floquet
from tqdm import tqdm

plt.rcParams.update({
    "text.usetex": True,
})
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.fallback'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'
plt.style.use('seaborn-v0_8-deep')
prop_cycle = plt.rcParams['axes.prop_cycle']
dcolors = prop_cycle.by_key()['color']

# Model

The O'brian-Fendley model is much like the normal Majorana-Hubbard chain but with a slightly modified interaction term:

\begin{align*}
H_{OF} =& -it\sum_j \gamma_j \gamma_{j+1} -g \sum_j \gamma_{j-2}\gamma_{j-1}\gamma_{j+1}\gamma_{j+2}\\
=& t \sum_j \left(\sigma_j^x +\sigma_j^z \sigma_{j+1}^z\right)
+g \sum_j\left(\sigma_j^x \sigma_{j+1}^x \sigma_{j+2}^z + \sigma_j^z \sigma_{j+1}^z \sigma_{j+2}^z\right)
\end{align*}

In [ ]:
def drive(t, omega):
    return np.cos(omega*t)

def Hof_dynamic(omega, g, basis):
    """
    Hopping strength is cos(omega t) (time t is fed later to the Hamiltonian function)
    g is the coupling strength
    basis is quspin spinless_fermion_basis_1d
    """
    
    L = basis.N
    
    x_lst = [[-2, i] for i in range(L)]
    
    zz_lst = [[-2, i, i+1] for i in range(L-1)]
    
    xzz_lst = [[g, i, i+1, i+2] for i in range(L-2)]
    zzx_lst = [[g, i, i+1, i+2] for i in range(L-2)]

    zz_lst += [[-2, L-1, 0]]
    
    xzz_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    zzx_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    static = [['xzz', xzz_lst],
              ['zzx', zzx_lst]]
    dynamic = [['x', x_lst, drive, [omega]],
               ['zz', zz_lst, drive, [omega]]]
    
    H = hamiltonian(static, dynamic, basis=basis, check_symm=False, check_herm=False)
    return H

# Time evolution

I'm going to start with the $t=0$ ground state and evolve for a few periods using a built-in function from QuSpin that handles dynamic Hamiltonians by solving the Schrodinger equation.

In [ ]:
L = 4
basis = spin_basis_1d(L, pauli=1) # pauli matrix, not spin 1/2 (matches iTensor "S=1/2")
g = 1
#omega = 1/np.sqrt(1+g**2)
omega = 100
H = Hof_dynamic(omega, g, basis)
e, v = H.eigh(time=0) # diagonalizing at time t=0
print(e[:10]) # +L*(t**2+g**2)/g)
v0 = v[:,0]
T = 2*np.pi/omega

In [ ]:
periods = 40
steps = 20*periods + 1
times = np.linspace(0, periods*T, steps)
energies = np.zeros(steps)
olaps = np.zeros(steps, dtype=np.complex128)
vs = H.evolve(v0, 0, times, iterate=True)
for i, vi in tqdm(enumerate(vs), total=steps): # tqdm just makes a progress bar. Feel free to replace this line with enumerate(vs):
    t = times[i]
    #print(f"t/2 pi = {np.round(t/(2*np.pi), 3)}")
    energies[i] = H.expt_value(vi, time=t).real
    olaps[i] = np.vdot(vi, v0)
    #print(f"e = {energies[i]}, |overlap| = {np.abs(olaps[i])}")

# Floquet spectrum, etc.

QuSpin provides code to comput the Floquet unitary $\mathcal U_F = \mathcal T_t \exp \left(-i\int_0^T d t H(t)\right)$ where $\mathcal T_t \exp$ is the time-ordered exponential.
I'm going to investigate what it's doing here.

In [ ]:
L = 4
basis = spin_basis_1d(L, pauli=1)
g = 1
omega = 100
T = 2*np.pi/omega
H = Hof_dynamic(omega, g, basis)

I'm kind of blindly copying the example from the docs here. This should compute the Floquet quasi-energies ```EF```, Hamiltonian ```HF```, evolution operator ```UF```, and eigenvasis ```VF```.

In [ ]:
n_periods = 1
t = Floquet_t_vec(omega, n_periods, len_T=10)

t_list = t.T *np.arange(0, 1, 1/100) + np.finfo(float).eps # times to evaluate H

dt_list = np.array([t.T/100 for ti in t_list]) # time step durations to apply H for

Floq = Floquet(
    {"H": H, "t_list": t_list, "dt_list": dt_list}, HF=True, VF=True, UF=True
)  # call Floquet class

EF = np.sort(Floq.EF)
HF = Floq.HF
UF = Floq.UF
VF = Floq.VF

In [ ]:
print(EF)

According to the docs, it's also possible to do a continuous-time version of this via solving some ODEs up to relative tolerance ```rtol```. It seems to be quite slow, so beware!

In [ ]:
Floq_c = Floquet({"H": H, 
                "T": T,
                "rtol": 1e-9})

In [ ]:
EF_c = np.sort(Floq_c.EF)
print(EF_c)

These are decently close! The errors seem to be roughly $(\delta t)^2$ for timestep $\delta t$ in the discretized case.

In [ ]:
N = 41
olaps_F = np.zeros(N, np.complex128)
energies_F = np.zeros(N)
e, v = H.eigh(time=0)
v0 = v[:,0]
vi = v0.copy()
for i in range(N):
    energies_F[i] = H.expt_value(vi, time=T*i).real
    olaps_F[i] = np.vdot(v0, vi) 
    vi = UF.dot(vi) # applying Floquet operator to get to next period

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=200)
ax.plot(np.arange(N), energies_F, 'o', color=dcolors[0], markerfacecolor='none')
ax.plot(times/T, energies, 'o', color=dcolors[0], markersize=1)

ax.set_xlabel(r'$t/T$')
ax.set_ylabel(r'$\langle\Psi(t)|H(t)|\Psi(t)\rangle$')# , color=dcolors[0])
ax.tick_params(axis='y', colors=dcolors[0])

ax2 = ax.twinx()
ax2.plot(np.arange(N), np.abs(olaps_F), 'o', color=dcolors[2], markerfacecolor='none')
ax2.plot(times/T, np.abs(olaps), 'o', color=dcolors[2], markersize=1)
ax2.set_ylabel(r'$|\langle \Psi(0)|\Psi(t)\rangle|$') #, color=dcolors[2])
ax2.tick_params(axis='y', colors=dcolors[2])
ax2.set_ylim(-.02, 1.02)

# plt.savefig("floquet_test.pdf", bbox_inches="tight")

This looks good: Floquet and direct time evolution agree at each period. 